In [ ]:
# pip install numpy
#yes u need to individually import the packages


In [ ]:
# pip install pandas

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies= pd.read_csv('E:/projects_I_will_never_do/pp/peronal-projects/ML_recommendation_system/datasets/tmdb_5000_movies.csv') #using absolute path coz me dumb
credits= pd.read_csv('E:/projects_I_will_never_do/pp/peronal-projects/ML_recommendation_system/datasets/tmdb_5000_credits.csv')

In [ ]:
# movies.head(1)

In [ ]:
movies= movies.merge(credits,on='title')

In [ ]:
# movies.head(1)

In [ ]:
 #important coloumns: genres, id,, keywords, title ,overview, vote_avg, vote_count, cast, crew
 movies= movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
# movies['original_language'].value_counts()
# movies.info()


In [ ]:
# we make a tag column by combining overview,genres, keywords, cast and crew columns
# so firstly we need to start preprocessing

In [ ]:
movies.isnull().sum() #check for null
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum() #check for duplicate

In [ ]:
movies.iloc[0].genres #need to convert this into action,advendture,fantasy,scifi

In [ ]:
pip install ast 


In [ ]:
import ast #imported for the function, literal eval, to evaluate string representation of a list

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])   #goes to each element of list, which is a dictionary and takes out the value of name key in the dictionary
    return L

In [ ]:
movies['genres']= movies['genres'].apply(convert) #converted the format of column, noice

In [ ]:
movies['keywords']= movies['keywords'].apply(convert) #and another

In [ ]:
def convert2(obj):
    L=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])   #goes to each element of list, which is a dictionary and takes out the value of name key in the dictionary
            counter+=1
        else: break
    return L

In [ ]:
movies['cast']=movies['cast'].apply(convert2)

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])   #goes to each element of list, which is a dictionary and takes out the value of name key in the dictionary
    return L

In [ ]:
movies['crew']= movies['crew'].apply(fetch_director)

In [ ]:
movies['overview']= movies['overview'].apply(lambda x: x.split())

In [ ]:
# now to remove spaces from keywords,cast and crew so that it doesn't mess up the model
movies['genres']= movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords']= movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast']= movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew']= movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
#now combining all of them
movies['tags']= movies['overview']+ movies['genres']+ movies['keywords']+movies['cast']+movies['crew']

In [ ]:
new_df= movies[['movie_id','title','tags']]

In [ ]:
#now conver the tags list to a string with space
new_df['tags']= new_df['tags'].apply(lambda x: " ".join(x))



In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

In [ ]:
#now vectorizing the damned thing ffs, basically plotting each tag on graph and when a user asks for recommendation we give him the closest calue to the vector
#gonna use BAG OF WORDS BABY
#and ofc we are gonna ignore the stop words


In [ ]:
pip install scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv= CountVectorizer(max_features= 5000, stop_words= 'english')

In [ ]:
pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()

In [ ]:
#before vectorizing we also stem that shit
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
vectors= cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names() #to check what words came in the most

In [ ]:
vectors[0]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity #basically to check the dist b/w each vector, lesser the dist more the similiarity

In [ ]:
similarity= cosine_similarity(vectors)

In [ ]:
#alright my head is starting to hurt

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x: x[1])

In [ ]:
def recommend(movie):
    movie_index= new_df[new_df['title'] == movie].index[0]
    distances= similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        # print(i[0])

    return

In [ ]:
recommend('Avatar') #ain't working properly atm, check it later

In [ ]:
pip install pickle


In [ ]:
import pickle

In [264]:
pickle.dump(new_df.to_dict(),open('movied.pkl','wb'))

In [266]:
pickle.dump(similarity,open('similarity.pkl','wb'))